In [51]:
!pip install sentence-transformers -q
!pip install --quiet -U pinecone-client
!pip install unidecode -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 15.4 MB/s eta 0:00:00


In [52]:
import pandas as pd
import numpy as np
import pinecone
import itertools
from unidecode import unidecode

products = pd.read_csv('https://raw.githubusercontent.com/ashwinkadam/DigitalMarketing-Algorithms-Project/main/Product_recom/products.csv')

In [53]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


In [ ]:
pinecone.init(api_key= '5d4c8961-5aaa-4aa5-a104-50561a86de58', environment='eu-west1-gcp')
index_name = 'products-sim'

if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=384)
index = pinecone.Index(index_name=index_name)

all_items = [str(title) for title in products['name']]
all_items = [unidecode(title) for title in all_items]

encoded_vector = model.encode(all_items)

items_to_insert = list(zip(all_items, encoded_vector.tolist()))

def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

print('Index statistics before upsert:', index.describe_index_stats())

for e, batch in enumerate(chunks([(ii[:64],x) for ii,x in items_to_insert])):index.upsert(vectors=batch)

print('Index statistics after upsert:', index.describe_index_stats())

In [75]:
product = "Tripp Black And Blue Strap Pants"

# create the query vector
xq = model.encode(product).tolist()

# now query
xc = index.query(xq, top_k=10, include_metadata=True)


df = pd.DataFrame({'Product': [result['id'] for result in xc['matches']],'score': [round(result['score'], 2) for result in xc['matches']]})[df['score'] != 1]
df

,Product,score
1,Tripp Black And Purple Chain Pants,0.86
2,Tripp Black And White Chain Pants,0.86
3,Tripp Black And Grey Mesh Pants,0.85
4,Tripp NYC Baggy Step Chain Pants in Red/Black ...,0.77
5,Tripp NYC Skinny T-Jean Pants in Black/White C...,0.76
6,Tripp NYC Skinny T-Jean Pants in Red/Black Che...,0.75
7,Tripp Red Plaid Zip Skinny Pants,0.75
8,Black Cotton Ripstop BDU Pants,0.71
9,RD Style Black Riding Style Legging Pants,0.70
